# Linear Regression

Los modelos de regresión son como 'calculadoras inteligentes' que usan datos pasados para predecir números. Por esta razón usaremos un ejemplo basado en un estudio real en el que se recopilaron datos de un sistema de bicicletas compartidas para predecir el número de alquileres en función de la estacionalidad y las condiciones climáticas. Utilizaremos una versión simplificada del conjunto de datos, que te presento a continuacion:

In [1]:
import pandas as pd

# carga el conjunto de datos de entrenamiento
bike_data = pd.read_csv('daily-bike-share.csv')
bike_data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals
0,1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331
1,2,1/2/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131
2,3,1/3/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120
3,4,1/4/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108
4,5,1/5/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82


Este es un dataset muy usado en cursos introductorios de machine learning, especialmente en entornos Microsoft (como Azure ML), y contiene datos históricos de uso de bicicletas compartidas.

### Contenido del dataset daily-bike-share.csv

Este dataset contiene registros diarios del número de bicicletas rentadas en un sistema de bike sharing, junto con variables meteorológicas y temporales.

Columna	Descripción
- **day**	Fecha en formato YYYY-MM-DD.
- **season**	Estación del año (1: invierno, 2: primavera, 3: verano, 4: otoño).
- **year**	0 para 2011, 1 para 2012.
- **month**	Número del mes (1-12).
- **holiday**	1 si es día festivo, 0 si no.
- **weekday**	Día de la semana (0: domingo, …, 6: sábado).
- **workingday**	1 si es día laboral, 0 si es fin de semana o feriado.
- **weather**	Estado del clima (1: despejado, 2: nublado, 3: lluvia ligera, 4: tormenta).
- **temp**	Temperatura normalizada (valor entre 0 y 1).
- **atemp**	Sensación térmica normalizada.
- **hum**	Humedad relativa normalizada.
- **windspeed**	Velocidad del viento normalizada.
- **rentals**	Número real de bicicletas rentadas ese día.

In [2]:
# Selecciona las columnas numéricas normalizadas de temperatura, sensación térmica, humedad y velocidad del viento
bike_data[['temp', 'atemp', 'hum', 'windspeed']]

,temp,atemp,hum,windspeed
0,0.344167,0.363625,0.805833,0.160446
1,0.363478,0.353739,0.696087,0.248539
2,0.196364,0.189405,0.437273,0.248309
3,0.200000,0.212122,0.590435,0.160296
4,0.226957,0.229270,0.436957,0.186900
...,...,...,...,...
726,0.254167,0.226642,0.652917,0.350133
727,0.253333,0.255046,0.590000,0.155471
728,0.253333,0.242400,0.752917,0.124383
729,0.255833,0.231700,0.483333,0.350754


# Variables normalizadas

- **temp**
- **atemp**
- **hum**
- **windspeed**

## ¿Qué significa que esté normalizada entre 0 y 1?

La normalización es una técnica de escalado de valores, usada comúnmente en machine learning para que todos los datos estén en un rango comparable, fórmula:

$$
\text{valor normalizado} = \frac{t - t_{\text{min}}}{t_{\text{max}} - t_{\text{min}}}
$$

donde:  
- Para **temp**: $t_{\text{min}} = -8^\circ C$, $t_{\text{max}} = 39^\circ C$  
- Para **atemp**: $t_{\text{min}} = -16^\circ C$, $t_{\text{max}} = 50^\circ C$

Esta información está respaldada por la descripción del dataset en la [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset).

En este conjunto de datos, 'rentals' (alquileres) representa la etiqueta (el valor y) que nuestro modelo debe aprender a predecir. Las otras columnas son características potenciales (valores x). Siempre existira la posibilidad de enriquecer este conjunto de datos.

Por ejemplo podemos realizar ingeniería de características (feature engineering ) para combinar o derivar nuevas características. Por ejemplo, agreguemos una nueva columna llamada day al DataFrame extrayendo el componente del día de la columna existente dteday. La nueva columna representa el día del mes, del 1 al 31.

In [16]:
# Agrega una nueva columna 'day' al DataFrame extrayendo el día del mes de la columna 'dteday'
bike_data['day'] = pd.DatetimeIndex(bike_data['dteday']).day

# Muestra las primeras 32 filas para verificar la nueva columna
bike_data.head(32)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals,day
0,1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,1
1,2,1/2/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,2
2,3,1/3/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,3
3,4,1/4/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,4
4,5,1/5/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,5
5,6,1/6/2011,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.089565,88,6
6,7,1/7/2011,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,7
7,8,1/8/2011,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,68,8
8,9,1/9/2011,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.361950,54,9
9,10,1/10/2011,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,10


Bien, comencemos nuestro análisis de los datos examinando algunas estadísticas descriptivas clave. Podemos usar el método describe del DataFrame para generarlas tanto para las características numéricas como para la columna de etiqueta rentals.

In [17]:
# Selecciona las características numéricas y la columna de etiquetas, y muestra estadísticas descriptivas
numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
bike_data[numeric_features + ['rentals']].describe()

,temp,atemp,hum,windspeed,rentals
count,731.000000,731.000000,731.000000,731.000000,731.000000
mean,0.495385,0.474354,0.627894,0.190486,848.176471
std,0.183051,0.162961,0.142429,0.077498,686.622488
min,0.059130,0.079070,0.000000,0.022392,2.000000
25%,0.337083,0.337842,0.520000,0.134950,315.500000
50%,0.498333,0.486733,0.626667,0.180975,713.000000
75%,0.655417,0.608602,0.730209,0.233214,1096.000000
max,0.861667,0.840896,0.972500,0.507463,3410.000000


Las estadísticas revelan información sobre la distribución de los datos en cada uno de los campos numéricos, incluyendo el número de observaciones (hay 731 registros), la media, la desviación estándar, los valores mínimos y máximos, y los valores de los cuartiles (los valores límite para el 25%, 50% —que también es la mediana— y el 75% de los datos). A partir de esto, podemos ver que el número medio de rentas diarias es de alrededor de 848; pero hay una desviación estándar relativamente alta, lo que indica mucha variabilidad en el número de rentas por día.

Podríamos obtener una idea más clara de la distribución de los valores de rentas visualizando los datos. Los tipos de gráficos más comunes para visualizar distribuciones de datos numéricos son los histogramas y los diagramas de caja (box plots), así que usaremos la biblioteca matplotlib de Python para crear uno de cada tipo para la columna rentals.

In [18]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

label = bike_data['rentals']

# Histograma
hist = go.Histogram(x=label, nbinsx=100, name='Rentals', marker_color='lightblue')

# Líneas para la media y la mediana
mean_line = go.Scatter(x=[label.mean(), label.mean()], y=[0, 100], mode='lines', name='Mean', line=dict(color='magenta', dash='dash'))
median_line = go.Scatter(x=[label.median(), label.median()], y=[0, 100], mode='lines', name='Median', line=dict(color='cyan', dash='dash'))

# Diagrama de caja (boxplot)
box = go.Box(x=label, name='Rentals', boxpoints='outliers', marker_color='orange', orientation='h')

# Crea subgráficos (subplots)
fig = make_subplots(rows=2, cols=1, subplot_titles=('Rental Distribution Histogram', 'Rental Distribution Boxplot'))

fig.add_trace(hist, row=1, col=1)
fig.add_trace(mean_line, row=1, col=1)
fig.add_trace(median_line, row=1, col=1)
fig.add_trace(box, row=2, col=1)

fig.update_layout(height=700, showlegend=True, title_text='Distribución de alquiler (Histograma y diagrama de caja)')
fig.update_xaxes(title_text='Rentals', row=2, col=1)
fig.update_yaxes(title_text='Frequency', row=1, col=1)

fig.show()

![Distribución de Rentas](assets/RentalDistribution.png)

Los gráficos muestran que el número de rentas diarias varía desde 0 hasta poco más de 3,400. Sin embargo, la media (y la mediana) del número de rentas diarias está más cerca del extremo inferior de ese rango, con la mayoría de los datos entre 0 y alrededor de 2,200 rentas. Los pocos valores por encima de esto se muestran en el diagrama de caja como pequeños círculos, lo que indica que son valores atípicos; en otras palabras, valores inusualmente altos o bajos que están fuera del rango típico de la mayoría de los datos.

Podemos hacer el mismo tipo de exploración visual con las características numéricas. Vamos a crear un histograma para cada una de ellas.

In [19]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Creamos una figura con 2 filas y 2 columnas para los histogramas
fig = make_subplots(rows=2, cols=2, subplot_titles=numeric_features)

# Recorremos cada característica numérica para graficar su histograma
for i, col in enumerate(numeric_features):
    feature = bike_data[col]
    row = i // 2 + 1  # Calcula la fila del subplot
    col_pos = i % 2 + 1  # Calcula la columna del subplot

    # Histograma de la característica
    hist = go.Histogram(
        x=feature, nbinsx=100, name=col, marker_color='lightblue', showlegend=False
    )
    # Línea de la media
    mean_line = go.Scatter(
        x=[feature.mean(), feature.mean()],
        y=[0, feature.value_counts().max()],
        mode='lines',
        name='Mean',
        line=dict(color='magenta', dash='dash'),
        showlegend=(i == 0)  # Solo muestra la leyenda una vez
    )
    # Línea de la mediana
    median_line = go.Scatter(
        x=[feature.median(), feature.median()],
        y=[0, feature.value_counts().max()],
        mode='lines',
        name='Median',
        line=dict(color='cyan', dash='dash'),
        showlegend=(i == 0)
    )
    # Agrega los trazos al subplot correspondiente
    fig.add_trace(hist, row=row, col=col_pos)
    fig.add_trace(mean_line, row=row, col=col_pos)
    fig.add_trace(median_line, row=row, col=col_pos)

# Ajusta el diseño y muestra la figura
fig.update_layout(height=700, width=1000, title_text='Histogramas de características numéricas')
fig.show()

![Distribución de Rentas](assets/Histograms.png)

Las características numéricas parecen estar distribuidas de forma más normal, con la media y la mediana más cercanas al centro del rango de valores, coincidiendo con los valores que ocurren con mayor frecuencia.

> **Nota**: Las distribuciones no son realmente normales en el sentido estadístico, lo cual implicaría un histograma suave y simétrico en forma de “campana”, con la media y la moda (el valor más común) en el centro; pero en general indican que la mayoría de las observaciones tienen un valor cercano al centro.

Ya hemos explorado la distribución de los valores numéricos en el conjunto de datos, pero ¿qué hay de las características categóricas? Estas no son números continuos en una escala, por lo que no podemos usar histogramas; pero sí podemos graficar un diagrama de barras que muestre la cantidad de veces que aparece cada valor discreto en cada categoría.

In [20]:
import plotly.graph_objects as go

categorical_features = ['season', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit', 'day']

for col in categorical_features:
    counts = bike_data[col].value_counts().sort_index()
    fig = go.Figure(data=[go.Bar(x=counts.index, y=counts.values, marker_color='steelblue')])
    fig.update_layout(
        title=f'{col} counts',
        xaxis_title=col,
        yaxis_title='Frequency'
    )
    fig.show()


![Distribución de Rentas](assets/DistributionsCategorical.png)

Muchas de las características categóricas muestran una distribución más o menos *uniforme* (es decir, hay aproximadamente la misma cantidad de filas para cada categoría). Las excepciones a esto incluyen:

- **holiday**: Hay muchos menos días festivos que días que no lo son.
- **workingday**: Hay más días laborables que no laborables.
- **weathersit**: La mayoría de los días pertenecen a la categoría *1* (despejado), siendo la categoría *2* (neblina y nubes) la siguiente más común. Hay relativamente pocos días en la categoría *3* (lluvia o nieve ligera), y ningún día en la categoría *4* (lluvia fuerte, granizo o niebla densa).

Ahora que sabemos algo sobre la distribución de los datos en nuestras columnas, podemos comenzar a buscar relaciones entre las características y la etiqueta **rentals** que queremos predecir.

Para las características numéricas, podemos crear diagramas de dispersión (*scatter plots*) que muestren la intersección entre los valores de las características y los valores de la etiqueta. También podemos calcular la estadística de *correlación* para cuantificar la relación aparente.

In [21]:
import plotly.graph_objects as go

for col in numeric_features:
    feature = bike_data[col]
    label = bike_data['rentals']
    correlation = feature.corr(label)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=feature,
        y=label,
        mode='markers',
        marker=dict(color='royalblue', opacity=0.6),
        name='Data'
    ))
    fig.update_layout(
        title=f'rentals vs {col} - correlation: {correlation:.3f}',
        xaxis_title=col,
        yaxis_title='Bike Rentals',
        height=500,
        width=700
    )
    fig.show()

![Diagramas de dispersión: Alquileres vs. Características numéricas](assets/RentalsvsNumericFeatures.png)

Los resultados no son concluyentes, pero si observas detenidamente los diagramas de dispersión para **temp** y **atemp**, puedes notar una tendencia diagonal tenue que indica que los conteos más altos de rentas tienden a coincidir con temperaturas más elevadas. Un valor de correlación ligeramente superior a 0.5 en ambas características respalda esta observación.

Por el contrario, los gráficos para **hum** (humedad) y **windspeed** (velocidad del viento) muestran una correlación ligeramente negativa, lo que indica que hay menos rentas en días con alta humedad o viento.

Ahora comparemos las características categóricas con la etiqueta. Haremos esto creando diagramas de caja (*box plots*) que muestren la distribución de los conteos de rentas para cada categoría.

In [22]:
import plotly.express as px

for col in categorical_features:
    fig = px.box(bike_data, x=col, y='rentals', points='outliers', color=col, title=f'Rentals by {col}')
    fig.update_layout(yaxis_title='Bike Rentals', xaxis_title=col)
    fig.show()

![Distribución de rentas por variable categórica](assets/DistributionRentalsCategoricalVariable.png)

Los gráficos muestran cierta variación en la relación entre algunos valores categóricos y el número de rentas. Por ejemplo, hay una diferencia clara en la distribución de rentas durante los fines de semana (**weekday** 0 o 6) en comparación con los días laborables (**weekday** del 1 al 5). De manera similar, también se observan diferencias notables en las categorías **holiday** y **workingday**.

Existe una tendencia evidente que muestra distintas distribuciones de rentas en los meses de primavera y verano en comparación con los meses de invierno y otoño. La categoría **weathersit** también parece influir en la distribución de rentas.

Por otro lado, la característica **day**, que creamos para representar el día del mes, muestra poca variación, lo que indica que probablemente **no es un buen predictor** del número de rentas.

## Entrenar un Modelo de Regresión

Ahora que hemos explorado los datos, es momento de usarlos para entrenar un modelo de regresión que utilice las características que hemos identificado como potencialmente predictivas para predecir la etiqueta **rentals**.

Lo primero que necesitamos hacer es separar las características que queremos usar para entrenar el modelo de la etiqueta que queremos que el modelo prediga.

In [23]:
# Separa las características (features) y las etiquetas (labels)
X, y = bike_data[['season','mnth', 'holiday','weekday','workingday','weathersit','temp', 'atemp', 'hum', 'windspeed']].values, bike_data['rentals'].values
print('Features:',X[:10], '\nLabels:', y[:10], sep='\n')

Features:
[[1.        1.        0.        6.        0.        2.        0.344167
  0.363625  0.805833  0.160446 ]
 [1.        1.        0.        0.        0.        2.        0.363478
  0.353739  0.696087  0.248539 ]
 [1.        1.        0.        1.        1.        1.        0.196364
  0.189405  0.437273  0.248309 ]
 [1.        1.        0.        2.        1.        1.        0.2
  0.212122  0.590435  0.160296 ]
 [1.        1.        0.        3.        1.        1.        0.226957
  0.22927   0.436957  0.1869   ]
 [1.        1.        0.        4.        1.        1.        0.204348
  0.233209  0.518261  0.0895652]
 [1.        1.        0.        5.        1.        2.        0.196522
  0.208839  0.498696  0.168726 ]
 [1.        1.        0.        6.        0.        2.        0.165
  0.162254  0.535833  0.266804 ]
 [1.        1.        0.        0.        0.        1.        0.138333
  0.116175  0.434167  0.36195  ]
 [1.        1.        0.        1.        1.        1.        

Después de separar el conjunto de datos, ahora tenemos arreglos de NumPy llamados **X** que contienen las características y **y** que contienen las etiquetas.

*Podríamos* entrenar un modelo utilizando todos los datos, pero en el aprendizaje supervisado es una práctica común dividir los datos en dos subconjuntos: un conjunto (normalmente más grande) para entrenar el modelo, y un conjunto más pequeño de "retención" para validar el modelo entrenado. Esto nos permite evaluar qué tan bien se desempeña el modelo al usar el conjunto de validación, comparando las etiquetas predichas con las etiquetas reales conocidas.

Es importante dividir los datos de manera *aleatoria* (en lugar de, por ejemplo, tomar el primer 70 % para entrenamiento y el resto para validación). Esto ayuda a asegurar que los dos subconjuntos de datos sean estadísticamente comparables (de modo que validemos el modelo con datos que tengan una distribución estadística similar a los datos con los que fue entrenado).

Para dividir los datos aleatoriamente, usaremos la función **train_test_split** de la biblioteca **scikit-learn**. Esta biblioteca es uno de los paquetes de aprendizaje automático más ampliamente utilizados en Python.

In [24]:
from sklearn.model_selection import train_test_split

# Divide los datos en un 70% para entrenamiento y un 30% para prueba de forma aleatoria
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

Training Set: 511 rows
Test Set: 220 rows


Ahora tenemos los siguientes cuatro conjuntos de datos:

- **X_train**: Los valores de características que usaremos para entrenar el modelo  
- **y_train**: Las etiquetas correspondientes que usaremos para entrenar el modelo  
- **X_test**: Los valores de características que usaremos para validar el modelo  
- **y_test**: Las etiquetas correspondientes que usaremos para validar el modelo

Ahora estamos listos para entrenar un modelo ajustando un algoritmo de regresión adecuado a los datos de entrenamiento. Usaremos un algoritmo de *regresión lineal*, un punto de partida común para problemas de regresión que funciona tratando de encontrar una relación lineal entre los valores de *X* y la etiqueta *y*. El modelo resultante es una función que, conceptualmente, define una línea donde se intersectan todas las combinaciones posibles de valores de X y y.

En Scikit-Learn, los algoritmos de entrenamiento están encapsulados en *estimadores*, y en este caso, usaremos el estimador **LinearRegression** para entrenar un modelo de regresión lineal.

In [25]:
# Entrena el modelo
from sklearn.linear_model import LinearRegression

# Ajusta un modelo de regresión lineal usando el conjunto de entrenamiento
model = LinearRegression().fit(X_train, y_train)
# print(model)
# print(model.get_params())
print(f"LinearRegression(copy_X={model.copy_X}, fit_intercept={model.fit_intercept}, n_jobs={model.n_jobs}, positive={model.positive})")

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, positive=False)


### Evaluar el Modelo Entrenado

Ahora que hemos entrenado el modelo, podemos usarlo para predecir los conteos de rentas utilizando las características que reservamos en nuestro conjunto de datos de validación. Luego podemos comparar estas predicciones con los valores reales de las etiquetas para evaluar qué tan bien (¡o no!) está funcionando el modelo.

In [26]:
import numpy as np

predictions = model.predict(X_test)
np.set_printoptions(suppress=True)
print('Predicted labels: ', np.round(predictions)[:10])
print('Actual labels   : ' ,y_test[:10])

Predicted labels:  [1896. 1184. 1007.  -28.  314.  385.  475.  590. 1476.  -22.]
Actual labels   :  [2418  754  222   47  244  145  240  555 3252   38]


Comparar cada predicción con su correspondiente valor real ("ground truth") no es una forma muy eficiente de determinar qué tan bien está prediciendo el modelo. Veamos si podemos obtener una mejor indicación visualizando un diagrama de dispersión (*scatter plot*) que compare las predicciones con las etiquetas reales.

También superpondremos una línea de tendencia para obtener una idea general de qué tan bien se alinean las etiquetas predichas con las verdaderas.

In [27]:
import numpy as np

import plotly.graph_objects as go

# Scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=y_test,
    y=predictions,
    mode='markers',
    marker=dict(color='royalblue', opacity=0.6),
    name='Predicciones'
))

# Línea de regresión
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
x_line = np.linspace(min(y_test), max(y_test), 100)
fig.add_trace(go.Scatter(
    x=x_line,
    y=p(x_line),
    mode='lines',
    line=dict(color='magenta'),
    name='Regresión'
))

fig.update_layout(
    title='Daily Bike Share Predictions',
    xaxis_title='Actual Labels',
    yaxis_title='Predicted Labels',
    height=500,
    width=700
)
fig.show()

Hay una tendencia diagonal clara, y las intersecciones entre los valores predichos y los valores reales siguen en general la trayectoria de la línea de tendencia; sin embargo, hay una diferencia considerable entre la función ideal representada por la línea y los resultados reales. Esta variación representa los *residuales* del modelo; es decir, la diferencia entre la etiqueta predicha cuando el modelo aplica los coeficientes aprendidos durante el entrenamiento a los datos de validación, y el valor real de la etiqueta de validación.

Estos residuales, cuando se evalúan con los datos de validación, indican el nivel esperado de *error* cuando el modelo se use con nuevos datos cuya etiqueta es desconocida.

Puedes cuantificar los residuales calculando algunas métricas de evaluación comúnmente utilizadas. Nos enfocaremos en las siguientes tres:

- **Error Cuadrático Medio (MSE)**: La media de las diferencias al cuadrado entre los valores predichos y los reales. Produce una métrica relativa: cuanto menor sea el valor, mejor se ajusta el modelo.
- **Raíz del Error Cuadrático Medio (RMSE)**: La raíz cuadrada del MSE. Proporciona una métrica absoluta en la misma unidad que la etiqueta (en este caso, número de rentas). Cuanto menor sea el valor, mejor es el modelo (de forma simple, representa el número promedio de rentas por el cual las predicciones se equivocan).
- **Coeficiente de determinación (conocido comúnmente como *R-cuadrado* o R²)**: Una métrica relativa en la que un valor más alto indica un mejor ajuste del modelo. En esencia, representa cuánta de la varianza entre los valores predichos y reales puede explicar el modelo.

> **Nota**: Puedes encontrar más información sobre estas y otras métricas de evaluación para modelos de regresión en la [documentación de Scikit-Learn](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

Vamos a usar Scikit-Learn para calcular estas métricas para nuestro modelo, basándonos en las predicciones generadas para el conjunto de validación.

In [28]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_test, predictions)
print("R2:", r2)

MSE: 201972.55947035612
RMSE: 449.4135728595167
R2: 0.6040454736919186


## Interpretación:

- **MSE (Error Cuadrático Medio): 201,972.56**  
  Representa el promedio de los cuadrados de las diferencias entre los valores reales y los valores predichos por el modelo. Un valor más bajo indica un mejor ajuste, pero su magnitud depende de la escala de la variable objetivo.

- **RMSE (Raíz del Error Cuadrático Medio): 449.41**  
  Es la raíz cuadrada del MSE, por lo que está en la misma unidad que la variable objetivo (número de rentas). En este caso, el modelo se equivoca en promedio por unas **449 rentas** al predecir el valor diario. Cuanto menor sea este valor, mejor será el desempeño del modelo.

- **R² (Coeficiente de determinación): 0.60**  
  Indica que el modelo explica aproximadamente el **60 %** de la variabilidad observada en el número de rentas. Un valor de 1 representa un ajuste perfecto; un valor de 0 significa que el modelo no explica nada de la variabilidad.

**En resumen:**  
El modelo tiene un poder predictivo razonable, pero aún hay margen de mejora. El error promedio es de unas 449 rentas y el modelo explica el 60 % de la variabilidad de los datos reales. Esto sugiere que, aunque el modelo capta tendencias generales, existen otros factores que podrían ayudar a mejorar la precisión de las predicciones.

## Lectura adicional

Para aprender más sobre Scikit-Learn, consulta la [documentación de Scikit-Learn](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics).